## POS tagging using modified Viterbi

### Data Preparation

In [245]:
#Importing libraries
import nltk

In [246]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [247]:
nltk_data[:5]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

In [248]:
# converting the list of sents to a list of (word, pos tag) tuples
tagged_words = [tup for sent in nltk_data for tup in sent]
print(len(tagged_words))
tagged_words[:10]

100676


[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET')]

In [249]:
# Splitting into train and test
import random
from sklearn.model_selection import train_test_split
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:2])

3718
196
[[('They', 'PRON'), ('argue', 'VERB'), ('that', 'ADP'), ('U.S.', 'NOUN'), ('investors', 'NOUN'), ('often', 'ADV'), ('can', 'VERB'), ('buy', 'VERB'), ('American', 'ADJ'), ('depositary', 'ADJ'), ('receipts', 'NOUN'), ('on', 'ADP'), ('the', 'DET'), ('big', 'ADJ'), ('stocks', 'NOUN'), ('in', 'ADP'), ('many', 'ADJ'), ('funds', 'NOUN'), (';', '.'), ('these', 'DET'), ('so-called', 'ADJ'), ('ADRs', 'NOUN'), ('represent', 'VERB'), ('shares', 'NOUN'), ('of', 'ADP'), ('foreign', 'ADJ'), ('companies', 'NOUN'), ('traded', 'VERB'), ('*', 'X'), ('in', 'ADP'), ('the', 'DET'), ('U.S.', 'NOUN'), ('.', '.')], [('Mr.', 'NOUN'), ('Stearn', 'NOUN'), (',', '.'), ('who', 'PRON'), ('*T*-196', 'X'), ('had', 'VERB'), ('been', 'VERB'), ('with', 'ADP'), ('the', 'DET'), ('company', 'NOUN'), ('more', 'ADJ'), ('than', 'ADP'), ('20', 'NUM'), ('years', 'NOUN'), ('and', 'CONJ'), ('had', 'VERB'), ('been', 'VERB'), ('president', 'NOUN'), ('since', 'ADP'), ('1984', 'NUM'), (',', '.'), ('will', 'VERB'), ('act', 'VE

In [250]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95610

In [251]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:5]

['They', 'argue', 'that', 'U.S.', 'investors']

In [252]:
# vocabulary
V = set(tokens)
print(len(V))

12075


In [253]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

### Build the vanilla Viterbi based POS tagger

In [254]:
import numpy as np
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))


In [255]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [256]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [257]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [258]:
import pandas as pd
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [259]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [260]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [261]:
tagged_seq_viterbi = Viterbi(test_tagged_words)

### Solve the problem of unknown words

### First solution by handling all unknowns as nouns

In [262]:
# Viterbi Heuristic with handle for unknown words
def Viterbi_handle_unknown(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    viterbi_expression = list(zip(words, state))
    handled_viterbi_expression = []
    #identifying words that are not there in the training set and assigning noun tag since most of the nouns will be missed
    for word, state in viterbi_expression:
        if word in tokens:
            handled_viterbi_expression.append((word,state))
        else:
            handled_viterbi_expression.append((word,'NOUN'))
    return handled_viterbi_expression

In [263]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [264]:
tagged_seq_viterbi_handle_unknown = Viterbi_handle_unknown(test_tagged_words)

### Second solution by using combination of lexicon and rule based

#### Lexicon(Unigram) Tagger

In [265]:
# Lexicon (or unigram tagger)
unigram_tagger = nltk.UnigramTagger(train_set)
unigram_tagger.evaluate(test_set)

0.9030793525463877

#### Rule based Tagger

In [266]:
# specify patterns for tagging
# example from the NLTK book
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*ould$', 'VERB'),              # modals
    (r'.*\'s$', 'NOUN$'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
]

In [267]:
regexp_tagger = nltk.RegexpTagger(patterns)

In [268]:
regexp_tagger.evaluate(test_set)

0.3337939202526648

#### Evaluating tagging accuracy

### Accuracy of Vanila Viterbi Algorithm

In [269]:
# accuracy
check = [i for i, j in zip(tagged_seq_viterbi, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_viterbi)
accuracy

0.8833333333333333

### Accuracy of tagger with unknown words tagged as nouns as most of the nouns are wrongly tagged

In [270]:
# accuracy
check = [i for i, j in zip(tagged_seq_viterbi_handle_unknown, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq_viterbi_handle_unknown)
accuracy

0.9

### Accuracy of two taggers combined

In [271]:
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

# lexicon backed up by the rule-based tagger
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)

lexicon_tagger.evaluate(test_set)

0.9476904855902092

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

### Tagging Accuracy of multiple Algorithm

In [272]:
## Testing
from nltk.tokenize import word_tokenize
sentences = nltk.sent_tokenize(open('Test_sentences.txt').read())
for sentence in sentences:
    words = word_tokenize(sentence)
    tagged_seq_sent_viterbi = Viterbi(words)
    tagged_seq_sent_handle_unk = Viterbi_handle_unknown(words)
    tagged_seq_sent_lexicon = lexicon_tagger.tag_sents([words])
    print("Vanila Viterbi Algorithm: ",tagged_seq_sent_viterbi)
    print("-----------------------------------------------")
    print("Viterbi Algorithm with unknown handled: ",tagged_seq_sent_handle_unk)
    print("-----------------------------------------------")
    print("Lexicon tagger: ",tagged_seq_sent_lexicon)
    print("-----------------------------------------------")
    print("*************************************************************************************************")

Vanila Viterbi Algorithm:  [('Android', 'PRT'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'PRT'), ('.', '.')]
-----------------------------------------------
Viterbi Algorithm with unknown handled:  [('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
-----------------------------------------------
Lexicon tagger:  [[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]]
-----------------------------------------------
*************************************************************************************************
Vanila Viterbi Algorithm:  [('Android', 'PRT'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'

Vanila Viterbi Algorithm:  [('Show', 'NOUN'), ('me', 'PRON'), ('the', 'DET'), ('cheapest', 'ADJ'), ('round', 'NOUN'), ('trips', 'PRT'), ('from', 'ADP'), ('Dallas', 'NOUN'), ('to', 'PRT'), ('Atlanta', 'NOUN'), ('I', 'PRON'), ('would', 'VERB'), ('like', 'ADP'), ('to', 'PRT'), ('see', 'VERB'), ('flights', 'NOUN'), ('from', 'ADP'), ('Denver', 'NOUN'), ('to', 'PRT'), ('Philadelphia', 'NOUN'), ('.', '.')]
-----------------------------------------------
Viterbi Algorithm with unknown handled:  [('Show', 'NOUN'), ('me', 'PRON'), ('the', 'DET'), ('cheapest', 'ADJ'), ('round', 'NOUN'), ('trips', 'NOUN'), ('from', 'ADP'), ('Dallas', 'NOUN'), ('to', 'PRT'), ('Atlanta', 'NOUN'), ('I', 'PRON'), ('would', 'VERB'), ('like', 'ADP'), ('to', 'PRT'), ('see', 'VERB'), ('flights', 'NOUN'), ('from', 'ADP'), ('Denver', 'NOUN'), ('to', 'PRT'), ('Philadelphia', 'NOUN'), ('.', '.')]
-----------------------------------------------
Lexicon tagger:  [[('Show', 'NOUN'), ('me', 'PRON'), ('the', 'DET'), ('cheapest', '

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [273]:
## instances with different tags by the algorithm
from nltk.tokenize import word_tokenize
sentences = nltk.sent_tokenize(open('Test_sentences.txt').read())
for sent_id, sentence in enumerate(sentences):
    words = word_tokenize(sentence)
    tagged_seq_sent_viterbi = Viterbi(words)
    tagged_seq_sent_handle_unk = Viterbi_handle_unknown(words)
    tagged_seq_sent_lexicon = lexicon_tagger.tag_sents([words])
    print("Sentence ",sent_id+1," : ",sentence)
    print("Word , Algo1_tag , Algo2_tag , Algo3_tag")
    for idx, (word, tag) in enumerate(tagged_seq_sent_viterbi):
        print(word,tagged_seq_sent_viterbi[idx][1],tagged_seq_sent_handle_unk[idx][1],tagged_seq_sent_lexicon[0][idx][1])
    print("**********************************************************************************************")

Sentence  1  :  Android is a mobile operating system developed by Google.
Word , Algo1_tag , Algo2_tag , Algo3_tag
Android PRT NOUN NOUN
is VERB VERB VERB
a DET DET DET
mobile ADJ ADJ ADJ
operating NOUN NOUN NOUN
system NOUN NOUN NOUN
developed VERB VERB VERB
by ADP ADP ADP
Google PRT NOUN NOUN
. . . .
**********************************************************************************************
Sentence  2  :  Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
Word , Algo1_tag , Algo2_tag , Algo3_tag
Android PRT NOUN NOUN
has VERB VERB VERB
been VERB VERB VERB
the DET DET DET
best-selling ADJ ADJ ADJ
OS PRT NOUN NOUN
worldwide PRT NOUN NOUN
on ADP ADP ADP
smartphones PRT NOUN VERB
since ADP ADP ADP
2011 PRT NOUN NUM
and CONJ CONJ CONJ
on ADP ADP ADP
tablets NOUN NOUN NOUN
since ADP ADP ADP
2013 PRT NOUN NUM
. . . .
**********************************************************************************************
Sentence  3  :  Google and T

### Comparison between Vanila viterbi algorithm, Viterbi algorithm with unknown words handle and Ensemble of lexicon and rule based tagger

As we can see, we have a set of 9 sentences for testing the tagging accuracies of the three different taggers that's built as a part of this assignment.

Stats(Accuracy):
Vanila Viterbi Algorithm : 0.8833
Viterbi with handle for unknown words : 0.9
Ensemble of Lexicon and rule based tagger : 0.9476

Note: 

1) There accuracy scores are generated on a randomly generated test set, so the scores might not remain the same when the code is re-run, but rest assured, it will be in the same range of +/- 0.05.

2) Another observation to be made is how the performance of the tagger increases by switching from the Vanila Viterbi to the Viterbi with handle for unknown words to the ensemble of the Lexicon and the Rule Based tagger

This can be noticed in a few examples from the test set.


The format of the output in the above cell: 
The first line shows the sentence that is being tagged
The second line is the header of the table where the first column is the word, second column is the tag of the 1st algorithm, third column is the tag of the 2nd algorithm and the 4th column is the tag of the 3rd algorithm

Inferences:

1) Most of the nouns in the Vanila Viterbi are tagger wrong as the algorithm return the first tag in the emission list if the word is not found in the training data. For example, In sentence 1, Android is tagged as "PRT" but should have been tagged as a "NOUN". This is corrected by using the missing word handle or the ensemble of the rule based and lexicon tagger

2) There are a few incorrect tags assosciated since the accuracy cannot reach a 100%. But a noticable change is that the ensemble model has a better predictive ability based on the semantics of the sentence as seen in Sentence 9; The word arriving was tagged as a "PRT" with the 1st algorithm, "NOUN" with the 2nd algorithm but the 3rd was able to properly tag it as a verb